In [6]:
!pip install pywaffle 

In [1]:
import pandas as pd

df = pd.read_csv('archive\olist_customers_dataset.csv')
df.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [2]:
import pandas as pd

df = pd.read_csv('archive\olist_orders_dataset.csv')
df.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [5]:
import pandas as pd

df = pd.read_csv('archive\olist_order_payments_dataset.csv')
df.dtypes

order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
from pywaffle import Waffle
import matplotlib.pyplot as plt

# Charger les données
orders = pd.read_csv('archive\olist_orders_dataset.csv')
payments = pd.read_csv('archive\olist_order_payments_dataset.csv')
customers = pd.read_csv('archive\olist_customers_dataset.csv')

# Joindre les données
df = pd.merge(orders, payments, on='order_id')
df = pd.merge(df, customers, on='customer_id')

# Convertir les dates
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Calculer les métriques RFM
now = datetime.now()
df_rfm = df.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (now - x.max()).days,
    'order_id': 'nunique',
    'payment_value': 'sum'
})
df_rfm.rename(columns={'order_purchase_timestamp': 'recency', 'order_id': 'frequency', 'payment_value': 'monetary_value'}, inplace=True)

# Check the number of unique values in each column
unique_r = df_rfm['recency'].nunique()
unique_f = df_rfm['frequency'].nunique()
unique_m = df_rfm['monetary_value'].nunique()

# Adjust the number of bins accordingly
r_labels = range(1, unique_r+1)
f_labels = range(1, unique_f+1)
m_labels = range(1, unique_m+1)

# Adjust the qcut function to match the number of unique values
r_quintiles = pd.qcut(df_rfm['recency'], q=unique_r, labels=r_labels, duplicates='drop')
f_quintiles = pd.qcut(df_rfm['frequency'], q=unique_f, labels=f_labels, duplicates='drop')
m_quintiles = pd.qcut(df_rfm['monetary_value'], q=unique_m, labels=m_labels, duplicates='drop')

# Assigner des couleurs à chaque segment RFM
colors = ['#d11141', '#00b159', '#00aedb', '#f37735', '#ffc425']
color_map = dict(zip(df_rfm['R'].unique(), colors))

# Compter le nombre de clients dans chaque segment RFM
counts = df_rfm.groupby(['R', 'F', 'M']).size().reset_index(name='count')

# Créer le plot_waffle_chart
fig = plt.figure(
    FigureClass=Waffle, 
    rows=5, 
    columns=20, 
    values=counts['count'], 
    colors=[color_map[x] for x in counts['R']],
    labels=["{}{}{}".format(r, f, m) for r, f, m in counts[['R', 'F', 'M']].values],
    legend={'loc': 'upper left', 'bbox_to_anchor': (1.1, 1)},
    title={'label': 'Segments RFM', 'loc': 'center'},
    figsize=(12, 8)
)

plt.show()

ValueError: Bin labels must be one fewer than the number of bin edges